## Generate features on all test + train and get predictions
* Todo: seperate feature generation stage, and allow addition of CV models, etc'. 
* uses list of candidates from lightfm
    * preprocess moti's predictions using code in `transform_moti_preds.py`

In [1]:
# !pip install catboost shap

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
from __future__ import print_function, absolute_import

from catboost import Pool, cv, CatBoostClassifier, CatBoostRegressor,CatBoost
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras import Model
import gc
import matplotlib.pyplot as plt
# import shap
# shap.initjs()

from copy import deepcopy
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split, GroupShuffleSplit, GridSearchCV, GroupKFold
from tensorflow.keras.metrics import TopKCategoricalAccuracy, Precision, SparseTopKCategoricalAccuracy # @4
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# from sklearn.metrics import top_k_accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

import pandas as pd
import numpy as np
np.random.seed(0)
# from sklearn.linear_model import  LogisticRegressionCV
# from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# from pytorch_tabnet.tab_model import TabNetClassifier
# from pytorch_tabnet.pretraining import TabNetPretrainer
# from pytorch_tabnet.metrics import Metric
# import torch

import datetime as dt
d1 = dt.datetime(2015,12,30) # arbitrary min start date , use for converting checkin, checkout to numbers in nice format

%matplotlib inline

pd.set_option("display.max_columns", 90)

In [4]:
# ## https://www.tensorflow.org/guide/mixed_precision ## TF mixed precision - pytorch requires other setup
# from tensorflow.keras.mixed_precision import experimental as mixed_precision

# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_policy(policy)
# ## will need to correct in places, e.g.: 
# ## outputs = layers.Activation('softmax', dtype='float32', name='predictions')(x)

#### Features to add:
* Lag 
* Rank (popularity) of city, country (in general, +- given booker country)
* Count of hotel; user, trip size ? (may be leaky )
* Seasonal features - Holidays? , datetime

Aggregate feats:
* user changed country? last booking (lag 1) country change? 
* max/min/avg popularity rank of previous locations visited



We should create a dictionary of the rank, count, city/country etc' feats, so we can easily merge them when making more "negative" samples/feats for ranking.


* Consider using a df2 of df without dates + drop_duplicates, +- without user/trip id (After calcing that) .


Leaky or potentially leaky (Dependso n test set): 
* Target freq features - frequency of target city, given source county +- affiliate +- month of year +- given country (and interactions of target freq). 
    * Risk of leaks - depends of test data has temporal split or not. 
    * cartboost can do target encode, but this lets us do it for interactions, e.g. target city freq given the 2 countries and affiliate.
    * beware overfitting! 

In [5]:
max_epochs = 100
FASTRUN = False #True False

if FASTRUN:
    max_epochs = 20

In [6]:
SAVE_OUTPUTS = False ## save intermediate files to disk

TARGET_COL = "label"
# most basic categorical columns , without 'user_id', , 'utrip_id' ordevice_class - used for count encoding/filtering
BASE_CAT_COLS = ['city_id',  'affiliate_id', 'booker_country', 'hotel_country']

### features to get lags for. Not very robust. May want different feats for lags before -1
LAG_FEAT_COLS = ['city_id', 
#                  'device_class',
       'affiliate_id',
#                  'booker_country',
                 'hotel_country', 
       'duration', 'same_country', 
#                  'checkin_weekday',
       'checkin_week',
#         'checkout_weekday',
       'city_id_count', 'affiliate_id_count',
                 'hotel_country_count', 
#                  'checkin_week_count',
                 'city_id_rank_by_hotel_country',
#        'city_id_rank_by_booker_country', ## there are only 5 booker countries
                 'city_id_rank_by_affiliate',
#        'affiliate_id_rank_by_hotel_country',
#        'affiliate_id_rank_by_booker_country', 
                ]

CAT_FEAT_NAMES = ["booker_country", "device_class","affiliate_id",
                  "city_id","hotel_country",
#                   "utrip_id", ## changed
#                   "user_id", ## ? could use lower dim - depends on train/test overlap
                  "checkin_week",#"checkout_week",
#                     "checkin_weekday",
    "lag1_city_id","lag1_hotel_country","lag1_affiliate_id", #"lag1_device_class",
     "lag2_city_id","lag2_hotel_country",#"lag2_affiliate_id",#"lag2_device_class",
       "lag3_city_id","lag3_hotel_country", #"lag3_booker_country","lag3_affiliate_id","lag3_device_class",
                  "first_hotel_country","first_city_id"
                 ]

TRAIN_FILE_PATH = "booking_train_set.csv" #"/content/drive/MyDrive/booking_wisdom/booking_train_set.csv" #"booking_train_set.csv"

LIST_FILE_PATH =   "all_candidates_list.parquet"#"predictions_list_proc_v2.csv.gz"#"predictions_list_proc_v2.csv.gz"

TEST_FILE_PATH = "booking_test_set.csv" #"/content/drive/MyDrive/booking_wisdom/booking_train_set.csv" #"booking_train_set.csv"
# TEST_LIST_FILE_PATH =  "list_booking_train.csv.gz"#"/content/drive/MyDrive/booking_wisdom/list_booking_train.csv.gz" #"list_booking_train.csv.gz"


In [7]:
# https://stackoverflow.com/questions/33907537/groupby-and-lag-all-columns-of-a-dataframe
# https://stackoverflow.com/questions/62924987/lag-multiple-variables-grouped-by-columns
## lag features with groupby over many columns: 
def groupbyLagFeatures(df:pd.DataFrame,lag:[]=[1,2],group="utrip_id",lag_feature_cols=[]):
    """
    lag features with groupby over many columns
    https://stackoverflow.com/questions/62924987/lag-multiple-variables-grouped-by-columns"""
    if len(lag_feature_cols)>0:
        df=pd.concat([df]+[df.groupby(group)[lag_feature_cols].shift(x).add_prefix('lag'+str(x)+"_") for x in lag],axis=1)
    else:
         df=pd.concat([df]+[df.groupby(group).shift(x).add_prefix('lag'+str(x)+"_") for x in lag],axis=1)
    return df

def groupbyFirstLagFeatures(df:pd.DataFrame,group="user_id",lag_feature_cols=[]):
    """
    Get  first/head value lag-like of features with groupby over columns. Assumes sorted data!
    """
    if len(lag_feature_cols)>0:
        df=pd.concat([df]+[df.groupby(group)[lag_feature_cols].transform("first").add_prefix("first_")],axis=1)
    else:
#          df=pd.concat([df]+[df.groupby(group).first().add_prefix("first_")],axis=1)
        df=pd.concat([df]+[df.groupby(group).transform("first").add_prefix("first_")],axis=1)
    return df

######## Get n most popular items, per group
def most_popular(group, n_max=4):
    """Find most popular hotel clusters by destination
    Define a function to get most popular hotels for a destination group.

    Previous version used nlargest() Series method to get indices of largest elements. But the method is rather slow.
    Source: https://www.kaggle.com/dvasyukova/predict-hotel-type-with-pandas
    """
    relevance = group['relevance'].values
    hotel_cluster = group['hotel_cluster'].values
    most_popular = hotel_cluster[np.argsort(relevance)[::-1]][:n_max]
    return np.array_str(most_popular)[1:-1] # remove square brackets

from tensorflow.keras.metrics import top_k_categorical_accuracy
def top_4_accuracy(y_true, y_pred):
    """will only work if doing multiclass predictions"""
    return top_k_categorical_accuracy(y_true, y_pred, k=4)

## https://codereview.stackexchange.com/questions/149306/select-the-n-most-frequent-items-from-a-pandas-groupby-dataframe
# https://stackoverflow.com/questions/52073054/group-by-a-column-to-find-the-most-frequent-value-in-another-column
## can get modes (sorted)
# https://stackoverflow.com/questions/50592762/finding-most-common-values-with-pandas-groupby-and-value-counts
## df.groupby('tag')['category'].agg(lambda x: x.value_counts().index[0])
# https://stackoverflow.com/questions/15222754/groupby-pandas-dataframe-and-select-most-common-value
# source2.groupby(['Country','City'])['Short name'].agg(pd.Series.mode)


# def fit_cbr_model(loss_function, additional_params=None, train_pool=train_pool, test_pool=test_pool,
#                   CV=True,return_model=False):
#     """
#     Fit catboost ranking model, or CV (cross validate) and return scores ; with a given loss function
#     https://colab.research.google.com/github/catboost/tutorials/blob/master/ranking/ranking_tutorial.ipynb#scrollTo=RVNW0nowbtxH
#     >>>model = fit_model('PairLogit', {'custom_metric': ['PrecisionAt:top=10', 'RecallAt:top=4']})
    
#     """
#     parameters = deepcopy(default_parameters)
#     parameters['loss_function'] = loss_function
#     parameters['train_dir'] = loss_function
    
#     if additional_params is not None:
#         parameters.update(additional_params)
         
#     if CV:
#         scores = cv(train_pool,params=parameters, plot=True,fold_count=2)
#         scores = scores[scores.columns[~scores.columns.str.contains("std|pair",case=False)]]
#         display(scores.tail())
        
#     ## train model to get hacky top4 accuracy
#     model = CatBoost(parameters)
#     model.fit(train_pool, eval_set=test_pool, plot=False)        
#     y_test_pred = model.predict(X_test,prediction_type="Probability")[:,1]
#     print("Test:")
#     top4_acc = top4_accuracy(model,y_test_pred)
#     auc_score = roc_auc_score(y_true=y_test,y_score=y_test_pred)
#     print("Roc-AUC (test): {0:.4f}".format(100*auc_score))
 
#     if return_model:
#         return model

In [8]:
dtypes_dict = {
#     "utrip_id":"category",
#     "city_id":"int32",
#     "rank":"int16",
    "label": "int8",#"uint8",
    "affiliate_id":"int32",
#     "device_class":"category",
#     "booker_country":"string", ## category would save more memory but intereferes with feature engineering
#     "hotel_country":"string"
}

In [9]:
# pd.read_csv(TRAIN_FILE_PATH,usecols=["city_id"]).min() ## lowest normal num in train city_id is 2

In [10]:
if FASTRUN:
    df = pd.read_csv(TRAIN_FILE_PATH,
                     nrows=350_280,
                     index_col=[0],
                     parse_dates=["checkin","checkout"],infer_datetime_format=True, dtype=dtypes_dict)
else:
    df = pd.read_csv(TRAIN_FILE_PATH,
                     index_col=[0],
                     parse_dates=["checkin","checkout"],infer_datetime_format=True, dtype=dtypes_dict)

# ## list of candidate predictions (100 per utrip_id)
# df_list = pd.read_parquet(LIST_FILE_PATH,
# #                         usecols=[0,2,3], # skip city_id col
# #                         dtype=dtypes_dict
#                      )

# df_list = df_list.loc[df_list["rank"]<50]
# print("rank of labels (target =1):")
# # print(df_list.loc[df_list["label"]==1]["rank"].describe(percentiles=[.25,.5,.75,.9,.95,.99]))

# print(df_list.shape[0])

# assert df_list.drop_duplicates(["utrip_id","city_id"]).shape[0] == df_list.shape[0]

# if FASTRUN:
#     df_list = df_list.loc[df_list["rank"]<=25]
# #     df_list = df_list.loc[(df_list["rank"]<=12) | (df_list["label"]==1)]  
# else:
#     #### downsampole train data, less so for test data/candidates
#     # # ## HACK due to running out of mempory ! sample of data 
    
#     train_ids_all = list(df["utrip_id"].unique())
#     print("len(train_ids_all)",len(train_ids_all))
#     df_list_test = df_list.loc[~df_list["utrip_id"].isin(train_ids_all)]
    
#     ## temp: for faster iterations
#     df_list_test = df_list_test.loc[df_list_test["rank"]<= 100]
    
#     print("list test shape",df_list_test.shape)
#     df_list_train = df_list.loc[df_list["utrip_id"].isin(train_ids_all)]
#     print("train test shape",df_list_train.shape)
    
#     df_list_1 = df_list_train.loc[(df_list_train["rank"]<=60) | (df_list_train["label"]==1)]    
#     df_list_2 = df_list_train.loc[(df_list_train["rank"]>60) & (df_list_train["label"]==0)].groupby(["utrip_id"]).sample(n=15,random_state=42)
#     df_list = pd.concat([df_list_1,df_list_2,df_list_test])
#     del df_list_1,df_list_2,df_list_test,df_list_train
#     gc.collect()

# print("df list shape",df_list.shape[0])

# ### df_test has "missing" cities, hotel countries
#  ## replace missing cities with nans or -9
# ### df_test["city_id"] = df_test["city_id"].replace(0,np.nan)

# # df_list = df_list.loc[df_list["city_id"] >0]

# df_list.set_index("utrip_id",inplace=True)
                     
df.sort_values(["user_id","checkin"],inplace=True)
# df_list.set_index("utrip_id",inplace=True)


## add feature - score relative to rank (could add vs utrip_id - equiv to rank)
# df_list["score_norm"] = df_list["score"].div(df_list.groupby("rank")["score"].transform("mean"))#.round(3)


display(df)
# display(df_list)

/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id
1004862,29,2016-07-09,2016-07-11,47054,desktop,1601,Elbonia,Elbonia,29_1
1004863,29,2016-07-11,2016-07-13,34444,desktop,1601,Elbonia,Elbonia,29_1
1004864,29,2016-07-13,2016-07-16,12291,desktop,1601,Elbonia,Elbonia,29_1
1004865,29,2016-07-16,2016-07-18,16386,desktop,8132,Elbonia,Elbonia,29_1
897811,81,2016-05-15,2016-05-16,33665,desktop,9924,Elbonia,Elbonia,81_1
...,...,...,...,...,...,...,...,...,...
1072993,6258065,2016-04-19,2016-04-20,55044,mobile,9452,Gondal,Pullamawang,6258065_1
420479,6258087,2016-08-03,2016-08-04,17754,desktop,2436,Gondal,Gondal,6258087_1
420480,6258087,2016-08-04,2016-08-05,50073,desktop,2436,Gondal,Gondal,6258087_1
420481,6258087,2016-08-05,2016-08-06,11662,desktop,2436,Gondal,Gondal,6258087_1


In [11]:
# df_test = pd.read_csv(TEST_FILE_PATH, parse_dates=["checkin","checkout"],infer_datetime_format=True, dtype=dtypes_dict)
# ### note the missing city id in last rows
# print(df_test.shape)
# test_num_ids = df_test["utrip_id"].nunique()
# print("test_num_ids",test_num_ids)
# df_test["train"] = 0

# ### df_test has "missing" cities, hotel countries
# # df_list["city_id"] = df_list["city_id"].replace(0,-9)# np.nan ## replace missing cities with nans or -9
# df_test["city_id"] = df_test["city_id"].replace(0,np.nan) ## 0 are missing cities here

# df_test

In [12]:
### add features to be consistent with test set of row in trip, and total trips in trip
df["row_num"] = df.groupby("utrip_id")["checkin"].rank(ascending=True,pct=False).astype(int)
utrip_counts = df["utrip_id"].value_counts()
df["total_rows"] = df["utrip_id"].map(utrip_counts)

df["train"] = 1

df["total_rows"].describe()

count    1.166835e+06
mean     6.117143e+00
std      2.796383e+00
min      1.000000e+00
25%      4.000000e+00
50%      5.000000e+00
75%      7.000000e+00
max      4.800000e+01
Name: total_rows, dtype: float64

In [13]:
# ### stack train, test for now (easier feat eng)
# df = pd.concat([df,df_test])
print(df.shape[0])

df["last"] = (df["row_num"] ==df["total_rows"]).astype(int)
assert 0 == df[["row_num","total_rows","last"]].isna().sum().max()
# df[["row_num","total_rows","last"]].describe()

1166835


In [14]:
### Drop duplicate, consecutive rows
df = df.loc[(df[["city_id","utrip_id","booker_country","last"]].shift() != df[["city_id","utrip_id","booker_country","last"]]).max(axis=1)]
print(df.shape[0])

## add alt total rows as a diff, for marking cases where we dropped dupes.

utrip_counts_2 = df["utrip_id"].value_counts()
df["diff_total_rows"] = df["utrip_id"].map(utrip_counts_2)
df["diff_total_rows"] =df["total_rows"] -  df["diff_total_rows"]

df[["diff_total_rows","total_rows"]].describe()

1102338


,diff_total_rows,total_rows
count,1.102338e+06,1.102338e+06
mean,2.314553e-01,6.101682e+00
std,6.809714e-01,2.728969e+00
min,0.000000e+00,1.000000e+00
25%,0.000000e+00,4.000000e+00
50%,0.000000e+00,5.000000e+00
75%,0.000000e+00,7.000000e+00
max,4.200000e+01,4.800000e+01


In [15]:
### todo : add "time since start (min date in data / "duration")"  
### todo : "time since start of trip (min date per trip "duration")"  
df["duration"] = ((df["checkout"] - df["checkin"]).dt.days).astype(int)
df["same_country"] = (df["booker_country"]==df["hotel_country"]).astype(int)

df["checkin_day"] = df["checkin"].dt.day
df["checkin_weekday"] = df["checkin"].dt.weekday
df["checkin_week"] = df["checkin"].dt.isocalendar().week.astype(int) ## week of year
df["checkin_month"] = df["checkin"].dt.month
df["checkin_year"] = df["checkin"].dt.year-2015
df["checkin_day_of_year"] = df["checkin"].dt.dayofyear  # day of year - may overfit , but will allow learning holidays and other checkins on that date 

# df["checkin_quarter"] = df["checkin"].dt.quarter # relatively redundant but may be used for "id"

# df["checkout_weekday"] = df["checkout"].dt.weekday
df["checkout_week"] = df["checkout"].dt.isocalendar().week.astype(int) ## week of year
df["checkout_day"] = df["checkout"].dt.day ## day of month

## cyclical datetime embeddings
## drop originakl variables? 
## TODO:L add for other variables, +- those that we'll embed (week?)

# df['checkin_weekday_sin'] = np.sin(df["checkin_weekday"]*(2.*np.pi/7))
# df['checkin_weekday_cos'] = np.cos(df["checkin_weekday"]*(2.*np.pi/7))

# df['checkin_month_sin'] = np.sin((df["checkin_month"]-1)*(2.*np.pi/12))
# df['checkin_month_cos'] = np.cos((df["checkin_month"]-1)*(2.*np.pi/12))

df['checkin_week_sin'] = np.sin((df["checkin_week"]-1)*(2.*np.pi/53))
df['checkin_week_cos'] = np.cos((df["checkin_week"]-1)*(2.*np.pi/53))

df.drop(["checkin_month"],axis=1,errors="ignore")

### time since first date in dataset (i.,e index proxy)
min_date = df["checkin"].min()
df["time_elapsed_global"] = (df["checkin"] - min_date).dt.days.astype(int)

# #############
# # last number in utrip id - probably which trip number it is:
# df["utrip_number"] = df["utrip_id"].str.split("_",expand=True)[1].astype(int)

### encode string columns - must be consistent with test data 
### IF we can concat test with train, we can just do a single transformation  for the NON TARGET cols
# obj_cols_list = df.select_dtypes("O").columns.values
obj_cols_list = ['device_class','booker_country','hotel_country'] # we could also define when loading data, dtype

for c in obj_cols_list:
    df[c] = df[c].astype("category")
    df[c] = df[c].cat.codes.astype(int)


In [16]:
def map_rare_feats(df,col,thresh=2,replace_val=-1):
    ### replace rare variables (under 2 occurrences) with "-1" dummy
    affiliates_counts = df[col].value_counts()
#     print("before:", affiliates_counts)
    print("uniques",df[col].nunique())
    affiliates_counts = affiliates_counts.to_dict()
    # df[col] = df[col].where(df[col].apply(lambda x: x.map(x.value_counts()))>=3, -1)
    df[col] = df[col].where(df[col].map(affiliates_counts)>=thresh, replace_val)
    print("afterwards uniques",df[col].nunique())
#     df[col] = df[col].astype(int)
    return(df[col])

In [17]:
### replace rare cities - note that we must take this into account with target! e.g. drop and account for missings as mistake
df["city_id"] = map_rare_feats(df,"city_id",thresh=2,replace_val=-1)
print("before: #trips",df["utrip_id"].nunique())
print("after: #trips",df.loc[(df["last"]==1) & (df["city_id"]>=0)]["utrip_id"].nunique())

uniques 39901
afterwards uniques 27714
before: #trips 217686
after: #trips 215508


In [18]:
print(df.shape[0])
print("trips without dropping rare last targets",df.loc[df["last"]==1]["utrip_id"].nunique())
print("trips after dropping rare last targets",df.loc[(df["last"]==1) & (df["city_id"]>0)]["utrip_id"].nunique())
df = df.loc[~((df["last"]==1) & (df["city_id"]<0))]
print(df.shape[0])

1102338
trips without dropping rare last targets 217686
trips after dropping rare last targets 215508
1100160


In [19]:
# ### replace rare variables (under 2 occurrences) with "-1" dummy
# affiliates_counts = df["affiliate_id"].value_counts()
# print("before:", affiliates_counts)
# print("uniques",df["affiliate_id"].nunique())
# affiliates_counts = affiliates_counts.to_dict()
# # df["affiliate_id"] = df["affiliate_id"].where(df["affiliate_id"].apply(lambda x: x.map(x.value_counts()))>=3, -1)
# df["affiliate_id"] = df["affiliate_id"].where(df["affiliate_id"].map(affiliates_counts)>4, -1)
# df["affiliate_id"] = df["affiliate_id"].astype(int)

# # print("after\n",df["affiliate_id"].value_counts())
# print("affiliate_id nunique",df["affiliate_id"].nunique())

In [20]:
df.city_id.min()

-1

In [21]:
# ### duplicate of city, country id columns, as number (may be redundant - we already have count encoding ! )
# ### replace rare variables (under 2 occurrences) with "-1" dummy
# ## city id:
# affiliates_counts = df["city_id"].value_counts()
# # print("before:", affiliates_counts)
# print("uniques",df["city_id"].nunique())
# affiliates_counts = affiliates_counts.to_dict()
# df["numeric_city_id"] = df["city_id"].where(df["city_id"].map(affiliates_counts)>3, -1)
# df["numeric_city_id"] = df["numeric_city_id"].astype(int)

# ## hotel_country: (+ordinal encoding)
# affiliates_counts = df["hotel_country"].value_counts()
# # print("before:", affiliates_counts)
# print("uniques",df["hotel_country"].nunique())
# affiliates_counts = affiliates_counts.to_dict()
# df["numeric_hotel_country"] = df["hotel_country"].where(df["hotel_country"].map(affiliates_counts)>3, "")
# df["numeric_hotel_country"] = df["numeric_hotel_country"].astype("category").cat.codes.astype(int)

* Add first country, city visited in a trip. 
* Drop first row of a trip

In [22]:
"""
## add the "first" place visited/values
### nopte - will need to drop first row in trip, or impute nans when using this feature 

### first by user results in too much sparsity/rareness for our IDs purposes
df = groupbyFirstLagFeatures(df,group="utrip_id",lag_feature_cols=["hotel_country","city_id","duration","same_country",
                                                                   "affiliate_id", "checkin"]) 
## + first checkin date -> use for time delta feature
# df = df.loc[df["row_num"]>1] ## can't do yet, needed for lag features
print(df[["first_hotel_country","hotel_country","city_id","first_same_country", "first_affiliate_id"]].nunique())
df
"""

'\n## add the "first" place visited/values\n### nopte - will need to drop first row in trip, or impute nans when using this feature \n\n### first by user results in too much sparsity/rareness for our IDs purposes\ndf = groupbyFirstLagFeatures(df,group="utrip_id",lag_feature_cols=["hotel_country","city_id","duration","same_country",\n                                                                   "affiliate_id", "checkin"]) \n## + first checkin date -> use for time delta feature\n# df = df.loc[df["row_num"]>1] ## can\'t do yet, needed for lag features\nprint(df[["first_hotel_country","hotel_country","city_id","first_same_country", "first_affiliate_id"]].nunique())\ndf\n'

Add relative date/duration feature(s)
* Time since start of trip (first_checkin)

In [23]:
"""
df["total_trip_duration"] = ((df["checkin"] -df["first_checkin"]).dt.days).astype(int)

df["first_checkin_week"] = df["first_checkin"].dt.isocalendar().week.astype(int)
# df[["total_trip_duration","first_checkin","checkin"]]

### change type of date cols to more learnable format

df["checkin"] = (df["checkin"]-d1).dt.days.astype(int16)
df["checkout"] = (df["checkout"]-d1).dt.days.astype(int16)
df["first_checkin"] = (df["first_checkin"]-d1).dt.days.astype(int16)
## # (df["checkin"]-d1).astype('int64')//1e9
"""

'\ndf["total_trip_duration"] = ((df["checkin"] -df["first_checkin"]).dt.days).astype(int)\n\ndf["first_checkin_week"] = df["first_checkin"].dt.isocalendar().week.astype(int)\n# df[["total_trip_duration","first_checkin","checkin"]]\n\n### change type of date cols to more learnable format\n\ndf["checkin"] = (df["checkin"]-d1).dt.days.astype(int16)\ndf["checkout"] = (df["checkout"]-d1).dt.days.astype(int16)\ndf["first_checkin"] = (df["first_checkin"]-d1).dt.days.astype(int16)\n## # (df["checkin"]-d1).astype(\'int64\')//1e9\n'

In [24]:
##### Following aggregation features - would be best to use time window (sort data) to generate, otherwise they will LEAK! (e.g. nunique countries visited)

### count features (can also later add rank inside groups).
### Some may be leaks (# visits in a trip should use time window?) , and do users repeat? 
### can add more counts of group X time period (e.g. affiliate X month of year)
## alt way to get counts/freq :
# freq = df["city_id"].value_counts()
# df["city_id_count"] = df["city_id"].map(freq)
# print(df["city_id_count"].describe())

count_cols = [ 'city_id', 'hotel_country', 
             'user_id', 
#              "checkin_month", 'affiliate_id', "checkin_week",
#               "checkin_day_of_year"
             ]

for c in count_cols:
    df[f"{c}_count"] = df.groupby([c])["duration"].transform("size")
    
# ########################################################
# ## nunique per trip
# ### https://stackoverflow.com/questions/46470743/how-to-efficiently-compute-a-rolling-unique-count-in-a-pandas-time-series

# nunique_cols = [ 'city_id', 'hotel_country','affiliate_id', 
#                 'booker_country',"duration","checkin_weekday"]
# # df["nunique_booker_countries"] = df.groupby("utrip_id")["booker_country"].nunique()
# # df["nunique_hotel_country"] = df.groupby("utrip_id")["hotel_country"].nunique()
# for c in nunique_cols:
#     df[f"{c}_nunique"] = df.groupby(["utrip_id"])[c].transform("nunique")

# ### trips taken by user
# df["user_nunique_trips"] = df.groupby("user_id")["utrip_id"].transform("nunique")
# print(df.nunique())

# ########################################################
# ## get frequency/count feature's rank within a group - e.g. within a country (or affiliate) 
# ## add "_count" to column name to get count col name, then add rank col 

# ### ALT/ duplicate feat - add percent rank (instead or in addition)

# rank_cols = ['city_id','affiliate_id', 'hotel_country',  # 'booker_country',
# #  "checkin_month",
#              "checkin_week"
#             ]
# ### what is meaning of groupby and rank of smae variable by same var? Surely should be 1 / unary? 
# for c in rank_cols:
#     df[f"{c}_rank_by_hotel_country"] = df.groupby(['hotel_country'])[f"{c}_count"].transform("rank")
#     df[f"{c}_rank_by_affiliate"] = df.groupby(['affiliate_id'])[f"{c}_count"].transform("rank")
#     ### rank by month: potentially strong feature, but must avoid leaks + must be joined back by month
# #     df[f"{c}_rank_by_month"] = df.groupby(['checkin_month'])[f"{c}_count"].transform("rank") ## "city_id_rank_by_month" may leak
# ## there are just 5 booker countries.. 
# df["city_id_rank_by_booker_country"] = df.groupby(['booker_country'])[f"city_id_count"].transform("rank")
    
# df

In [25]:
df.loc[df["city_id_count"]>=7]["city_id"].nunique()

12173

In [26]:
# assert df.isna().sum().max() ==0

In [27]:
## check coverage of top k classes
y_last = df.loc[(df["last"]==1) & (df["train"]==1)]["city_id"].dropna().astype(int)
print("sum top4 total percentage:",100*y_last.value_counts(normalize=True)[0:4].sum().round(3)) ##5.7%
print("sum top200 total percentage:",100*y_last.value_counts(normalize=True)[0:200].sum().round(3)) ## 48%
print("sum top500 total percentage:",100*y_last.value_counts(normalize=True)[0:500].sum().round(3)) ### 61%
print("sum top1000 total percentage:",100*y_last.value_counts(normalize=True)[0:1000].sum().round(3)) ### 71 %

sum top4 total percentage: 5.800000000000001
sum top200 total percentage: 49.0
sum top500 total percentage: 62.4
sum top1000 total percentage: 72.1


### add lag features + Train/test/data split
* Lag feats (remember for categorical)
* Drop leak features (target values - country, city)

* drop instances  that lack history (e.g. at least 3d step and onwards) - by dropna in lag feat
* fill nans
* Split train/test by `user id` / split could maybe be by `utrip ID` ? ? 
    * Test - only last trip
    *  stratified train/test split by class - then drop any train rows with overlap with tests' IDs.  
        * Could also stratify by users, but risks some classes being non present in test

* TODO: aggregate features o ntime series - e.g. mean, max, min duration
    * We already have nunique
###### WE may not need to drop these features anymore - but we may want to join them by city id
*We  join city id features after adding the list of caNDIDATes

In [28]:
### features to drop - not usable, or leaks (e.g. aggregations on target) , or hotel/city specific (should be added from per city features)

DROP_FEATS = [ #'user_id', ## drop seperately
#     'checkin',  # is ok if turned into number
#     'checkout',
              'hotel_country',
              'city_id_count','same_country',
#               'utrip_id',
             'city_id_count','hotel_country_count',
              'city_id_nunique', 'hotel_country_nunique',
              'city_id_rank_by_hotel_country','city_id_rank_by_booker_country', 'city_id_rank_by_affiliate',
              'affiliate_id_rank_by_hotel_country','affiliate_id_rank_by_booker_country', 'affiliate_id_rank_by_affiliate',
              'hotel_country_rank_by_hotel_country',
       'hotel_country_rank_by_booker_country','hotel_country_rank_by_affiliate',
              'booker_country_rank_by_hotel_country',
              'booker_country_rank_by_booker_country',
              'checkin_month_rank_by_hotel_country',
             ]

# df2.drop(DROP_FEATS,axis=1).columns

In [29]:
df["city_id"].value_counts().describe()

count    27714.000000
mean        39.696904
std        245.509379
min          2.000000
25%          3.000000
50%          5.000000
75%         15.000000
max      10010.000000
Name: city_id, dtype: float64

In [30]:
df.loc[df["last"]==1]["city_id"].value_counts().describe()

count    17721.000000
mean        12.161165
std         82.974868
min          1.000000
25%          1.000000
50%          2.000000
75%          5.000000
max       3792.000000
Name: city_id, dtype: float64

### quick filter rare cities
* Doesn't necessarily filter target cities only
* should maybe use utrip id, not user id

In [31]:
# df = df.loc[df["city_id_count"]>=2]
print("before # trips",df["utrip_id"].unique())
s1 = df.shape[0]
print(s1)
df = df.loc[~((df["last"]==1) & (df["city_id"]<0))]
print("rows after dropping rare targets",df.shape[0])

keep_users = df.loc[df["last"]==1]["utrip_id"].unique() # list  of users who still have a last address
df = df.loc[df["utrip_id"].isin(keep_users)] ## keep only users who have last destination (After our filtering)

df["user_id_count"] = df.groupby(["utrip_id"])["city_id"].transform("size")
df = df.loc[df["user_id_count"]>=4]
##########
#filter by frequency of last city target
pre_trips_amount = len(keep_users)
last_targets_freq = df.loc[df["last"]==1]["city_id"].value_counts().reset_index().rename(columns={"index":"city_id","city_id":"counts"})
print(last_targets_freq["counts"].describe())
### min target frequency : 
raw_count = last_targets_freq.shape[0]
print("prefiltering #",raw_count)
last_targets_freq = last_targets_freq.loc[last_targets_freq["counts"]>= 4]
print("cities after filtering:",last_targets_freq.shape[0])
print("% kept",100*(last_targets_freq.shape[0]/raw_count))
print("% dropped",100*(1-last_targets_freq.shape[0]/raw_count))

df = df.loc[df["city_id"].isin(last_targets_freq["city_id"])]
#######
df["user_id_count"] = df.groupby(["utrip_id"])["city_id"].transform("size") ## update count
df = df.loc[df["user_id_count"]>=4]
#######
post_trips_amount = df["utrip_id"].nunique()
print("% trips kept",100*(post_trips_amount/post_trips_amount))
print("% trips dropped",100*(1-post_trips_amount/pre_trips_amount))
#######

print(df.shape[0])
print(df[["utrip_id","city_id"]].nunique())
print(df.drop_duplicates("utrip_id")[["user_id_count","city_id_count"]].describe() )

print("After # trips",df["utrip_id"].unique())

before # trips ['29_1' '81_1' '136_2' ... '6258041_1' '6258065_1' '6258087_1']
1100160
rows after dropping rare targets 1100160
count    17266.000000
mean        11.195413
std         73.359163
min          1.000000
25%          1.000000
50%          2.000000
75%          4.750000
max       3083.000000
Name: counts, dtype: float64
prefiltering # 17266
cities after filtering: 5264
% kept 30.487663616355842
% dropped 69.51233638364415
% trips kept 100.0
% trips dropped 30.98075245466526
771966
utrip_id    148742
city_id       5255
dtype: int64
       user_id_count  city_id_count
count  148742.000000  148742.000000
mean        5.189967    2211.680346
std         1.734815    2578.081411
min         4.000000       5.000000
25%         4.000000     291.000000
50%         5.000000    1074.000000
75%         6.000000    3404.000000
max        35.000000    9959.000000
After # trips ['136_2' '136_4' '156_1' ... '6257895_1' '6257933_3' '6258017_1']


In [32]:
post_trips_amount = df["utrip_id"].nunique()
print("% trips kept",100*(post_trips_amount/pre_trips_amount))
print("% trips dropped",100*(1-post_trips_amount/pre_trips_amount))

% trips kept 69.01924754533474
% trips dropped 30.98075245466526


### lbl encode city, users 
* reencoding is wasteful, but saves on bugs ? (e.g. for embedding, which assumes start from zero)
* We add 1 , due to using zero padding for empty lag features! 


## I later reencode the target city_id, to save on space/# layers
* this loses the original values for now! 
* this is due to  how the sparse categoircal acc works - and we had encoded city id by the full vocab before, which is more cities than exist as targets in the final dest (and that defines the softmax layer size) 
    * final_target_encoder = LabelEncoder()


In [33]:
df["utrip_id"].value_counts().describe()

count    148742.000000
mean          5.189967
std           1.734815
min           4.000000
25%           4.000000
50%           5.000000
75%           6.000000
max          35.000000
Name: utrip_id, dtype: float64

In [34]:
ord_encoder = OrdinalEncoder()
print("min",df[["utrip_id","city_id"]].min(),"max", df[["utrip_id","city_id"]].max())
df[["utrip_id","city_id"]] = ord_encoder.fit_transform(df[["utrip_id","city_id"]] )
df["city_id"] =  df["city_id"] + 1 ## start values at 1
print("min",df[["utrip_id","city_id"]].min(),"max", df[["utrip_id","city_id"]].max())


min utrip_id    1000033_1
city_id             7
dtype: object max utrip_id    999855_1
city_id        67560
dtype: object
min utrip_id    0.0
city_id     1.0
dtype: float64 max utrip_id    148741.0
city_id       5255.0
dtype: float64


In [35]:
vocab_size = int(df["city_id"].max()+1)
print("max city id",vocab_size)

max city id 5256


#### resume pipeline - lag features for city id only + fillna 0  (sequence) 

In [36]:
print("df.shape",df.shape)
# ### lag features - last n visits
### orig: 
# df_feat = groupbyLagFeatures(df=df.copy(), 
#                    lag=[1,2],group="utrip_id",lag_feature_cols=LAG_FEAT_COLS)

df_feat = groupbyLagFeatures(df=df.copy(), 
                   lag=[1,2,3,4,5],group="utrip_id",lag_feature_cols=["city_id"])

# df_feat = df_feat.dropna(subset=["lag2_city_id"]).sample(frac=1)
df_feat = df_feat.loc[df_feat["last"]==1].drop(["last","row_num"],axis=1,errors="ignore")
# df_feat = df_feat.drop(DROP_FEATS,axis=1,errors="ignore")
print("df_feat",df_feat.shape)
# assert df_feat["utrip_id"].nunique() == df["utrip_id"].nunique()

df.shape (771966, 30)
df_feat (144236, 33)


In [37]:
### keep just lag cols for basic model
MINIMAL_COLS = ['city_id','lag1_city_id', 'lag2_city_id',
       'lag3_city_id', 'lag4_city_id', 
                'lag5_city_id',
               "utrip_id"]
df_feat = df_feat[MINIMAL_COLS]
df_feat = df_feat.fillna(0)
df_feat = df_feat.apply(pd.to_numeric, errors='ignore',downcast="integer")
df_feat

,city_id,lag1_city_id,lag2_city_id,lag3_city_id,lag4_city_id,lag5_city_id,utrip_id
770550,3525,4711,2237,3030,0,0,9695
117283,3214,3543,3624,4375,4022,4128,9696
426973,4503,1513,3283,1308,189,0,15043
818963,1338,3034,368,4707,1338,0,38636
230216,2816,3525,4306,732,0,0,54419
...,...,...,...,...,...,...,...
909369,2916,5030,1226,661,0,0,138728
990966,4765,4931,1700,4026,3970,4831,138729
379898,3024,1690,294,2526,0,0,138731
1078894,1229,88,1229,88,0,0,138732


In [38]:
## reorder/encode target city id - sincei t has less values than full vocab
final_target_encoder = LabelEncoder()
df_feat["city_id"] = final_target_encoder.fit_transform(df_feat["city_id"])


# num_classes = df_feat["city_id"].nunique()+1 ## target outputs
num_classes = int(df_feat["city_id"].max())
print(num_classes)

5148


Save intermediate files

In [39]:
# if SAVE_OUTPUTS:
#     df_feat.to_parquet(path="df_merged_train.parquet",index=False)
#     df_candidates.to_parquet(path="df_merged_candidates.parquet",index=False)

In [40]:
########################
## Groupwise train/test split by user or utrip_id
TARGET_COL = "city_id" ## overwrite
train_inds, test_inds = next(GroupShuffleSplit(test_size=.25,
                                               n_splits=2,
                                               random_state = 12).split(df_feat,groups=df_feat['utrip_id']))

X_train = df_feat.iloc[train_inds].drop(DROP_FEATS+["utrip_id"],axis=1,errors="ignore")#.merge(df_city_features,on="city_id",how="left")
X_test = df_feat.iloc[test_inds].drop(DROP_FEATS +["utrip_id"],axis=1,errors="ignore")#.merge(df_city_features,on="city_id",how="left")

y_train = X_train.pop(TARGET_COL)
y_test = X_test.pop(TARGET_COL)

print("# classes",y_train.nunique())

# ## check that same classes in train and test - 
# assert (set(y_train.unique()) == set(y_test.unique()))
assert y_train.isna().max() == y_test.isna().max() == 0

display(X_train)

# classes 5036


,lag1_city_id,lag2_city_id,lag3_city_id,lag4_city_id,lag5_city_id
770550,4711,2237,3030,0,0
426973,1513,3283,1308,189,0
818963,3034,368,4707,1338,0
230216,3525,4306,732,0,0
720067,5080,3106,4138,0,0
...,...,...,...,...,...
1060933,524,1741,3803,0,0
909369,5030,1226,661,0,0
379898,1690,294,2526,0,0
1078894,88,1229,88,0,0


### Embedding + multiclass model
* apply self attention
* Shared embedding - apply same embedding weights to all entities - https://stackoverflow.com/questions/42461673/apply-a-shared-embedding-layer-on-a-set-of-documents-in-keras

* see also previous booking modelsL:
    * https://www.kaggle.com/parlin987p/booking-sample-cnn#Bidirectional-+-masking 
    * https://www.kaggle.com/danofer/booking-sample-lstm-fork-1
    * https://www.kaggle.com/danofer/booking-model-dan
    
Note - we could also add triplet/ ranking loss
    * https://github.com/maciejkula/triplet_recommendations_keras

In [41]:
import tensorflow_recommenders as tfrs
from tensorflow_recommenders.layers.dcn import Cross
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, AdditiveAttention, Attention, Dropout

callback =[ tf.keras.callbacks.EarlyStopping(patience=3),  #,monitor=“val_mae”,baseline=5
tf.keras.callbacks.ReduceLROnPlateau(factor=0.3, patience=2) ## we should reset the LR when finetuning ?
            ]

In [77]:
### basic embedding + maxpooling model - trivial , very fast
def pooling_model():
    x = Input(shape=(X_train.shape[1],1,),name="city_lags") ## dim set by # of lag features/cols
    ## apply same embedding to all city - time distributed 
    emb = Embedding(input_dim=vocab_size+1, output_dim=300, mask_zero=True)
    att = TimeDistributed(emb)(x)
    att = TimeDistributed(GlobalMaxPooling1D())(att)
    att = Flatten()(att)
    out = Dense(num_classes+1,activation="softmax")(att)
    model = Model(x, out)
    model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=[tf.keras.metrics.SparseTopKCategoricalAccuracy(k=4)])
    print(model.summary())
    return model

def self_attention_model_flat():
    inp = Input(shape=(X_train.shape[1],1,),name="city_lags") ## dim set by # of lag features/cols
    ## apply same embedding to all city - time distributed 
    emb = Embedding(input_dim=vocab_size+1, output_dim=300, mask_zero=True)

    x = TimeDistributed(emb)(inp)
#     x = Flatten()(x) # orig
    x = TimeDistributed(Flatten())(x) ## alt     (seemingly no difference)
    att = Cross()(x)
    att = Cross()(x,att)
    att = Flatten()(att)
    out = Dense(num_classes+1,activation="softmax")(att)
    model = Model(inp, out)
    model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=[tf.keras.metrics.SparseTopKCategoricalAccuracy(k=4)])
    print(model.summary())
    return model


def self_cross_model_dense():
    inp = Input(shape=(X_train.shape[1],1,),name="city_lags") ## dim set by # of lag features/cols
    ## apply same embedding to all city - time distributed 
    emb = Embedding(input_dim=vocab_size+1, output_dim=300)

    x = TimeDistributed(emb)(inp)
    x = Cross()(x)
#     x = Flatten()(x) # orig
    x = TimeDistributed(Flatten())(x) ## alt     
    att = Cross()(x)
    att = Cross()(x,att)
    att = Flatten()(att)
    
#     C = concatenate([att, x ### add raw inputs +- embedding - semi residual/skip. 
#                  ])
    x = Dense(1024, activation="relu")(att)
    x = BatchNormalization()(x)
    x = Dense(2048, activation="relu")(x)
    x = BatchNormalization()(x)
    out = Dense(num_classes+1,activation="softmax")(x)
    model = Model(inp, out)
    model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=[tf.keras.metrics.SparseTopKCategoricalAccuracy(k=4)])
    print(model.summary())
    return model


def self_attention_flat():
    inp = Input(shape=(X_train.shape[1],1,),name="city_lags") ## dim set by # of lag features/cols
    ## apply same embedding to all city - time distributed 
    emb = Embedding(input_dim=vocab_size+1, output_dim=300, mask_zero=True)

    x = TimeDistributed(emb)(inp)
#     x = Flatten()(x) # orig
    x = TimeDistributed(Flatten())(x) ## alt     
    att = MultiHeadAttention(num_heads=4, key_dim=4)(x,x) # return_attention_scores=True
#     att = MultiHeadAttention(num_heads=4, key_dim=4)(att,att)

    att = Flatten()(att)
    out = Dense(num_classes+1,activation="softmax")(att)
    model = Model(inp, out)
    model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=[tf.keras.metrics.SparseTopKCategoricalAccuracy(k=4)])
    print(model.summary())
    return model

def self_attention_model_dense():
    inp = Input(shape=(X_train.shape[1],1,),name="city_lags") ## dim set by # of lag features/cols
    ## apply same embedding to all city - time distributed 
    emb = Embedding(input_dim=vocab_size+1, output_dim=300)

    x = TimeDistributed(emb)(inp)
#     x = Flatten()(x) # orig
    x = TimeDistributed(Flatten())(x) ## alt     
    att = MultiHeadAttention(num_heads=4, key_dim=4)(x,x)
    att = MultiHeadAttention(num_heads=4, key_dim=4)(att,x)

    att = Flatten()(att)
    att = LayerNormalization()(att)
#     C = concatenate([att, x ### add raw inputs +- embedding - semi residual/skip. 
#                  ])
    
    x = Dense(1024, activation="relu")(att)
    x = LayerNormalization()(x)
    x = Dense(1024, activation="relu")(x)
    att = LayerNormalization()(x)
    out = Dense(num_classes+1,activation="softmax")(att)
    model = Model(inp, out)
    model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=[tf.keras.metrics.SparseTopKCategoricalAccuracy(k=4)])
    print(model.summary())
    return model

def self_attention_model_2():
    inp = Input(shape=(X_train.shape[1],1,),name="city_lags") ## dim set by # of lag features/cols
    ## apply same embedding to all city - time distributed 
    emb = Embedding(input_dim=vocab_size+1, output_dim=300,
#                     mask_zero=True
                   )

    x = TimeDistributed(emb)(inp)
#     x = Flatten()(x) # orig
    x = TimeDistributed(Flatten())(x) ## alt     
    att = Attention(use_scale=True,dropout=0.4)([x,x])
    att = Attention(use_scale=True)([att,att])

    att = Flatten()(att)
    out = Dense(num_classes+1,activation="softmax")(att)
    model = Model(inp, out)
    model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=[tf.keras.metrics.SparseTopKCategoricalAccuracy(k=4)])
    print(model.summary())
    return model



def self_attention_model_dense_2():
    inp = Input(shape=(X_train.shape[1],1,),name="city_lags") ## dim set by # of lag features/cols
    ## apply same embedding to all city - time distributed 
    emb = Embedding(input_dim=vocab_size+1, output_dim=300)

    x1 = TimeDistributed(emb)(inp)
#     x = Flatten()(x) # orig
    max_pooled = Flatten()(TimeDistributed(GlobalMaxPooling1D())(x1))
    mean_pooled = Flatten()(TimeDistributed(GlobalAveragePooling1D())(x1))
    x = TimeDistributed(Flatten())(x1) ## alt     
    att = Attention(use_scale=True,dropout=0.25)([x,x])
    att = Attention(use_scale=True)([att,att])
    att = Flatten()(att)
#     att = LayerNormalization()(att)
    C = concatenate([att, max_pooled,mean_pooled ### add raw inputs +- embedding - semi residual/skip. 
                 ])
#     C = Flatten()(C)
    x = Dense(1024, activation="relu")(C)
    x = LayerNormalization()(x)
    x = Dense(1024, activation="relu")(x)
#     x = LayerNormalization()(x)
    out = Dense(num_classes+1,activation="softmax")(att)
    model = Model(inp, out)
    model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=[tf.keras.metrics.SparseTopKCategoricalAccuracy(k=4)])
    print(model.summary())
    return model

In [43]:
# ### could try copy sasREC keras code from : self-attention-recommendation  - it has dependnecies and failed for me (squeeze/dim bugs)
# ## https://github.com/KanchiShimono/self-attention-recommendation/blob/master/sasrec/sasrec.py
# # from /Users/oferd2/Documents/github/self-attention-recommendation-master import 
# import sys  
# sys.path.insert(0, '/Users/oferd2/Documents/github/self-attention-recommendation-master/sasrec/')
# ## dependencies of the sra 
# # ! pip install keras-self-attention keras-multi-head keras-pos-embd keras-position-wise-feed-forward keras-layer-normalization
# # export TF_KERAS=1 # error? 
# import os
# os.environ['TF_KERAS'] = "1"

# from sasrec import *
# # import sampler.py


In [44]:
X_train

,lag1_city_id,lag2_city_id,lag3_city_id,lag4_city_id,lag5_city_id
770550,4711,2237,3030,0,0
426973,1513,3283,1308,189,0
818963,3034,368,4707,1338,0
230216,3525,4306,732,0,0
720067,5080,3106,4138,0,0
...,...,...,...,...,...
1060933,524,1741,3803,0,0
909369,5030,1226,661,0,0
379898,1690,294,2526,0,0
1078894,88,1229,88,0,0


In [45]:
input_dim = vocab_size
import tensorflow
def self_attention_residual():
    """
    Based on self attention recc, with changes, and doesn't use positional or multihead, and may not do fc right? 
    see orig: SAR code: https://github.com/KanchiShimono/self-attention-recommendation 
    """
    #     inputs = Input(shape=(1,X_train.shape[1]),name="food") 
    inputs = keras.Input(shape=(X_train.shape[1]),name="food") 
    # inputs = Input(shape=(None,),name="food") 
    # inputs = keras.Input(shape=(None,))

    ## apply same embedding to all city - time distributed 
    # emb = Embedding(input_dim=input_dim, output_dim=64,
    #                 input_length=5
    #                )###Embedding(input_dim=vocab_size+1, output_dim=128)
    # #     x = TimeDistributed(emb)(inputs)
    # x = emb()(inputs)

    emd_dim = 128
    x = Embedding(input_dim=input_dim, output_dim=emd_dim,
                    input_length=5,
                  mask_zero=True,
                 )(inputs)
    ### following could be stacked into multiple blocks
    att = AdditiveAttention(use_scale=True,dropout=0.5)([x,x])
    att = Add()([att, x])
    feed_forward_input = tensorflow.keras.layers.LayerNormalization()(att)
    feed_forward_x = Dense(emd_dim,activation='relu',)(feed_forward_input)
    feed_forward_x = Dropout(0.4)(feed_forward_x)
    feed_forward_x = Add()([feed_forward_input, feed_forward_x])
    block_output = LayerNormalization(trainable=True)(feed_forward_x)
    
    ### opt: additional,residual attention block
    x = block_output
    att = AdditiveAttention(use_scale=True,dropout=0.5)([x,x]) 
    att = Add()([att, x])
    feed_forward_input = tensorflow.keras.layers.LayerNormalization()(att)
    feed_forward_x = Dense(emd_dim,activation='relu',)(feed_forward_input)
    feed_forward_x = Dropout(0.4)(feed_forward_x)
    feed_forward_x = Add()([feed_forward_input, feed_forward_x])
    block_output = LayerNormalization(trainable=True)(feed_forward_x)
       
    block_output = Flatten()(block_output)
    block_output = Dropout(0.1)(block_output)
    #     att = GlobalAveragePooling1D()(att)
    out = Dense(num_classes+1,activation="softmax")(block_output)
    model = Model(inputs, out)
    #     model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics=["accuracy",tf.keras.metrics.TopKCategoricalAccuracy(k=3)])
    model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam",
                  metrics=["accuracy",tf.keras.metrics.SparseTopKCategoricalAccuracy(k=4)])
#     print(model.summary())
    return(model)

model_sar = self_attention_residual()
hist_sar = model_sar.fit( X_train.values,#np.array(X_train.values.tolist()),
                         y_train,
                 batch_size=64,
                 validation_data=(X_test, y_test),
                 epochs=7,
                        callbacks= callback, verbose=1)

# y_pred =np.argmax(model_sar.predict(X_test), axis=-1)
# print("accuracy:{:.4f}".format(metrics.accuracy_score(y_true=y_test,y_pred=y_pred)))

#### .35 dropout, 300 embedding dim, 128 FC: 

    # ### with 1 block:
    # Epoch 5/5 - 32s 19ms/step - loss: 1.4337 - accuracy: 0.5882 - sparse_top_k_categorical_accuracy: 0.8592 
    #             - val_loss: 4.4571 - val_accuracy: 0.2870 - val_sparse_top_k_categorical_accuracy: 0.5228

    ### with 2 self attention/residual blocks : 
    ## after 1 epoch, val_accuracy: 0.29 - val_sparse_top_k_categorical_accuracy: 0.523. 
    ### perf dropped (like in prev) after that, on val data
    
    
    
#### .45, .25 dropout, 128 embedding/FC dim: 
## 
# Epoch 5/7 - 50s 30ms/step - loss: 1.9486 - accuracy: 0.4750 - sparse_top_k_categorical_accuracy: 0.7762 - val_loss: 4.1214 - val_accuracy: 0.2959 - val_sparse_top_k_categorical_accuracy: 0.5221

Epoch 1/7
1691/1691 [==============================] - 50s 29ms/step - loss: 5.1771 - accuracy: 0.2128 - sparse_top_k_categorical_accuracy: 0.3945 - val_loss: 3.8664 - val_accuracy: 0.2808 - val_sparse_top_k_categorical_accuracy: 0.5040
Epoch 2/7
1691/1691 [==============================] - 49s 29ms/step - loss: 3.3924 - accuracy: 0.2917 - sparse_top_k_categorical_accuracy: 0.5385 - val_loss: 3.7507 - val_accuracy: 0.2922 - val_sparse_top_k_categorical_accuracy: 0.5210
Epoch 3/7
1691/1691 [==============================] - 48s 28ms/step - loss: 2.8730 - accuracy: 0.3318 - sparse_top_k_categorical_accuracy: 0.6101 - val_loss: 3.8370 - val_accuracy: 0.2928 - val_sparse_top_k_categorical_accuracy: 0.5196
Epoch 4/7
1691/1691 [==============================] - 50s 30ms/step - loss: 2.4772 - accuracy: 0.3809 - sparse_top_k_categorical_accuracy: 0.6787 - val_loss: 4.0036 - val_accuracy: 0.2854 - val_sparse_top_k_categorical_accuracy: 0.5151
Epoch 5/7
1691/1691 [==============================]

In [46]:
input_dim = vocab_size
import tensorflow
def self_attention_residual():
    """
    Based on self attention recc, with changes, and doesn't use positional or multihead, and may not do fc right? 
    see orig: SAR code: https://github.com/KanchiShimono/self-attention-recommendation 
    """
    #     inputs = Input(shape=(1,X_train.shape[1]),name="food") 
    inputs = keras.Input(shape=(X_train.shape[1]),name="food") 

    emd_dim = 64
    x = Embedding(input_dim=input_dim, output_dim=emd_dim,
                    input_length=5,
                  mask_zero=True,
                 )(inputs)
    ### following could be stacked into multiple blocks
    att = AdditiveAttention(use_scale=True,dropout=0.5)([x,x])
    att = Add()([att, x])
    feed_forward_input = tensorflow.keras.layers.LayerNormalization()(att)
    feed_forward_x = Dense(emd_dim,activation='relu',)(feed_forward_input)
    feed_forward_x = Dropout(0.4)(feed_forward_x)
    feed_forward_x = Add()([feed_forward_input, feed_forward_x])
    block_output = tensorflow.keras.layers.LayerNormalization()(feed_forward_x)
       
    block_output = Flatten()(block_output)
    block_output = Dropout(0.1)(block_output)
    #     att = GlobalAveragePooling1D()(att)
    out = Dense(num_classes+1,activation="softmax")(block_output)
    model = Model(inputs, out)
    #     model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics=["accuracy",tf.keras.metrics.TopKCategoricalAccuracy(k=3)])
    model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam",
                  metrics=["accuracy",tf.keras.metrics.SparseTopKCategoricalAccuracy(k=4)])
#     print(model.summary())
    return(model)

model_sar = self_attention_residual()
hist_sar = model_sar.fit( X_train.values,#np.array(X_train.values.tolist()),
                         y_train,
                 batch_size=64,
                 validation_data=(X_test, y_test),
                 epochs=7,
                        callbacks= callback, verbose=1)

## single block, 
# Epoch 5/7 25s 15ms/step - loss: 1.8988 - accuracy: 0.4923 - sparse_top_k_categorical_accuracy: 0.7885 - val_loss: 4.0083 - val_accuracy: 0.3002 - val_sparse_top_k_categorical_accuracy: 0.5333
                        
## single att block,Attention (instead of AdditiveAttention) - slightly inferior: val_sparse_top_k_categorical_accuracy: 0.5293

Epoch 1/7
1691/1691 [==============================] - 25s 14ms/step - loss: 5.3114 - accuracy: 0.2175 - sparse_top_k_categorical_accuracy: 0.3836 - val_loss: 3.7152 - val_accuracy: 0.2951 - val_sparse_top_k_categorical_accuracy: 0.5198
Epoch 2/7
1691/1691 [==============================] - 24s 14ms/step - loss: 3.2655 - accuracy: 0.3112 - sparse_top_k_categorical_accuracy: 0.5622 - val_loss: 3.6519 - val_accuracy: 0.3019 - val_sparse_top_k_categorical_accuracy: 0.5273
Epoch 3/7
1691/1691 [==============================] - 24s 14ms/step - loss: 2.7188 - accuracy: 0.3600 - sparse_top_k_categorical_accuracy: 0.6422 - val_loss: 3.7661 - val_accuracy: 0.2942 - val_sparse_top_k_categorical_accuracy: 0.5269
Epoch 4/7
1691/1691 [==============================] - 26s 15ms/step - loss: 2.3371 - accuracy: 0.4086 - sparse_top_k_categorical_accuracy: 0.7102 - val_loss: 3.9591 - val_accuracy: 0.2940 - val_sparse_top_k_categorical_accuracy: 0.5270
Epoch 5/7
1691/1691 [==============================]

In [48]:
# ## transformer expect np.array, not multiple cols , so reshape
# ## https://stackoverflow.com/questions/54731199/how-to-feed-multiple-pandas-columns-into-keras-for-learning

# # X1 =  #np.array([np.array(l) for l in X])
# X1 = np.array(X_train.values)
# ## X1 = np.array(X_train.values.tolist())
# print(X1.shape)
# X1

# # X1 = np.array(X_train.values.tolist()).astype('float32')# .shape
# # print(X1.shape)
# # X1


# from tensorflow.keras.preprocessing.sequence import pad_sequences
# X1 = pad_sequences(X_train[cols].values.tolist())
# print(X1.shape)
# X1

(108177, 5)


array([[4711, 2237, 3030,    0,    0],
       [1513, 3283, 1308,  189,    0],
       [3034,  368, 4707, 1338,    0],
       ...,
       [1690,  294, 2526,    0,    0],
       [  88, 1229,   88,    0,    0],
       [4009,  561, 4407, 1955, 4523]], dtype=int16)

In [62]:
cols = X_train.columns

In [69]:
model = self_attention_model_2()

model.fit(X_train,y_train,
          epochs=15,
          callbacks=callback, ## lrplateau, early stopping
          validation_data=(X_test,y_test),
          batch_size = 64,
         )
### 54  val topk

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
city_lags (InputLayer)          [(None, 5, 1)]       0                                            
__________________________________________________________________________________________________
time_distributed_2 (TimeDistrib (None, 5, 1, 300)    1577100     city_lags[0][0]                  
__________________________________________________________________________________________________
time_distributed_3 (TimeDistrib (None, 5, 300)       0           time_distributed_2[0][0]         
__________________________________________________________________________________________________
attention_1 (Attention)         (None, 5, 300)       1           time_distributed_3[0][0]         
                                                                 time_distributed_3[0][0]   

In [73]:
# model = self_attention_flat()

# model.fit(X_train,y_train,
#           epochs=15,
#           callbacks=callback, ## lrplateau, early stopping
#           validation_data=(X_test,y_test),
#           batch_size = 64,
#          )

In [74]:
model = self_attention_model_dense()

model.fit(X_train,y_train,
          epochs=15,
          callbacks=callback, ## lrplateau, early stopping
          validation_data=(X_test,y_test),
          batch_size = 64,
         )


Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
city_lags (InputLayer)          [(None, 5, 1)]       0                                            
__________________________________________________________________________________________________
time_distributed_6 (TimeDistrib (None, 5, 1, 300)    1577100     city_lags[0][0]                  
__________________________________________________________________________________________________
time_distributed_7 (TimeDistrib (None, 5, 300)       0           time_distributed_6[0][0]         
__________________________________________________________________________________________________
multi_head_attention (MultiHead (None, 5, 300)       19548       time_distributed_7[0][0]         
                                                                 time_distributed_7[0][0]   

In [78]:
model = self_cross_model_dense()

model.fit(X_train,y_train,
          epochs=15,
          callbacks=callback, ## lrplateau, early stopping
          validation_data=(X_test,y_test),
          batch_size = 64,
         )


Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
city_lags (InputLayer)          [(None, 5, 1)]       0                                            
__________________________________________________________________________________________________
time_distributed_12 (TimeDistri (None, 5, 1, 300)    1577100     city_lags[0][0]                  
__________________________________________________________________________________________________
cross (Cross)                   (None, 5, 1, 300)    90300       time_distributed_12[0][0]        
__________________________________________________________________________________________________
time_distributed_13 (TimeDistri (None, 5, 300)       0           cross[0][0]                      
____________________________________________________________________________________________

In [79]:
model = pooling_model()

model.fit(X_train,y_train,
          epochs=15,
          callbacks=callback, ## lrplateau, early stopping
          validation_data=(X_test,y_test),
          batch_size = 64,
         )

# # Epoch 7/10 - 20s - loss: 1.907 - accuracy: 0.479 - sparse_top_k_categorical_accuracy: 0.789 -
# #             val_loss: 3.091 - val_accuracy: 0.3453 - val_sparse_top_k_categorical_accuracy: 0.5922

TypeError: Layer global_max_pooling1d does not support masking, but was passed an input_mask: Tensor("Reshape:0", shape=(None, 1), dtype=bool)

In [ ]:
# model = self_attention_model_flat_1()

# model.fit(X_train,y_train,
#           epochs=15,
#           callbacks=callback, ## lrplateau, early stopping
#           validation_data=(X_test,y_test),
#           batch_size = 64,
#          )

# # Epoch 2/10
# # 1527/1527 [==============================] - 23s 15ms/step - loss: 2.7761 - accuracy: 0.3593 - sparse_top_k_categorical_accuracy: 0.6269 - val_loss: 3.0526 - val_accuracy: 0.3387 - val_sparse_top_k_categorical_accuracy: 0.5939
# # Epoch 6/10
# # 1527/1527 [==============================] - 23s 15ms/step - loss: 1.3743 - accuracy: 0.5895 - sparse_top_k_categorical_accuracy: 0.8710 - val_loss: 3.8441 - val_accuracy: 0.3205 - val_sparse_top_k_categorical_accuracy: 0.5692

In [ ]:
## pooling, 2 epochs: 
# 5s 8ms/step - loss: 2.9698 - accuracy: 0.3515 - sparse_top_k_categorical_accuracy: 0.6059 - val_loss: 2.8055 - val_accuracy: 0.3643 - val_sparse_top_k_categorical_accuracy: 0.6204

In [ ]:
def top4_accuracy(model,y_test_pred=None):
    """warning - very hacky! - set to use global X_test, y_test variables"""
    if y_test_pred is None:
        y_test_pred = model.predict(X_test,prediction_type="Probability")[:,1]

    test_preds = pd.DataFrame({"y_test_pred":y_test_pred,
#                                "candidate_city_id":X_test["city_id"],
                               "utrip_id":X_test["utrip_id"],
                               "rank":X_test["rank"],
#                                "city_id_count":X_test["city_id_count"],
                               "label":y_test
                              })    
    ## most frequent in overall data (not as target)
    n_users = test_preds["utrip_id"].nunique()

#     test_preds.sort_values(["utrip_id","city_id_count"],inplace=True,ascending=False)
#     res_count = 100*test_preds.groupby("utrip_id")["label"].head(4).sum() / n_users
#     print("top4 acc baseline: by most frequent city_id_count {0:.4f}".format(res_count))

    test_preds.sort_values(["utrip_id","rank"],inplace=True,ascending=True)
    res_rank = 100*test_preds.groupby("utrip_id")["label"].head(4).sum() / n_users
    print("top4 acc baseline: by rank {0:.4f}".format(res_rank))

    test_preds.sort_values(["utrip_id","y_test_pred"],inplace=True,ascending=False)
    res = 100*test_preds.groupby("utrip_id")["label"].head(4).sum() / n_users
    print("top4 acc: by model {0:.4f}".format(res))
    return res

#### Get predictions on competition/candidate data
* `df_candidates`
* There appear to be duplicate city_ids in the predictions ? 

In [ ]:
%%time
if SAVE_OUTPUTS:
    candidates_pool = Pool(data=df_candidates,cat_features=CAT_FEAT_NAMES
#                      ,group_id=df_candidates["utrip_id"]
                     )
    print("candidate pool done")
    preds = model.predict(candidates_pool,prediction_type="Probability")[:,1]

In [ ]:
# %%time
# # errors if using raw data - data type disparities . unclear why
# ## thread_count
# preds = model.predict(df_candidates,prediction_type="Probability",verbose=True,thread_count=4)[:,1]
# print("done")
# print(len(preds))

In [ ]:
if SAVE_OUTPUTS:
    submission = pd.DataFrame({
                               "utrip_id":df_candidates["utrip_id"],
                           "city_id":df_candidates["city_id"],
                           "pred":preds,
    #                        "rank":df_candidates["rank"],
                          })   

    submission.sort_values(["pred"],ascending=False,inplace=True)

    ### unknonw bug - we get multiple cities per ID ???

    print("duplicate cities suggested:",submission.shape[0] - submission.drop_duplicates(['utrip_id','city_id']).shape[0])

    #  if SAVE_OUTPUTS:
    submission = submission.drop_duplicates(['utrip_id','city_id'],keep="first")
    print("submission.nunique()",submission.nunique())
    display(submission)
    ## drop -1s:
    submission = submission.loc[submission['city_id']>= 0]
    submission = submission.groupby(["utrip_id"]).head(4).drop("pred",axis=1)

    submission["r_num"] = submission.groupby("utrip_id")["city_id"].rank(ascending=True,pct=False)

    submission = submission.pivot(index='utrip_id', columns='r_num', values='city_id').reset_index() ## , values='score'
    submission.columns = ["utrip_id",'city_id_1','city_id_2','city_id_3','city_id_4']

    assert submission["utrip_id"].nunique() == test_num_ids
    display(submission)       

    submission.to_csv("submission.csv",index=False)
    print(submission.nunique())
    print(submission.shape)

In [ ]:
## gets stuck with many rows

# y_test_preds = model.predict(test_pool,prediction_type="Probability")[:,1]
# print(top4_accuracy(model,y_test_preds))

### from run on sample, with old train-train lightfm ranking: 
# top4 acc baseline: by most frequent city_id_count 39.6007
# top4 acc baseline: by rank 41.2646
# top4 acc: by model 77.0383
# 77.03826955074875

# ## still ~ 160k sample , model without rank/score: 
# top4 acc baseline: by most frequent city_id_count 32.1688
# top4 acc baseline: by rank 41.6647
# top4 acc: by model 49.9179

## Feature importance
* if usingSHAP: We don't want to wait too long/to calculate it over 40 million rows, so we could look at a sample. e.g. top ranked (rank <= 10 | label ==1)  , or first few rows


In [ ]:
top4_acc = top4_accuracy(model)
# # print(top4_accuracy(model))

# ## without sqrt balance, top4 acc: by model 56.9870

In [ ]:
# feat_imp_cb = model.get_feature_importance(data=test_pool,
#                        prettified=True,shap_calc_type ="Approximate")

# feat_imp_cb = feat_imp_cb.loc[feat_imp_cb["Importances"]>0].round(2)
# display(feat_imp_cb.head(32))

###### Evaluate model on test data
    * function may have memory leakage issue

* Using 30% of data:
    * 
        top4 acc baseline: by most frequent city_id_count 23.99
        
        top4 acc baseline: by rank 41.87
        
        top4 acc: by model 55.13